In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,694 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,133 kB]
Get:14 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InR

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-11 18:24:19--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.94MB/s    in 0.2s    

2022-04-11 18:24:20 (4.94 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

## Load Amazon Data into Spark Dataframe

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Baby_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

## Recreate the vine_table


In [5]:
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8EWA1OFT84NX|          5|            0|          0|   N|                Y|
|R2JWY4YRQD4FOP|          5|            0|          0|   N|                N|
| RL5ESX231LZ0B|          5|            0|          0|   N|                Y|
| RRMS9ZWJ2KD08|          5|            0|          0|   N|                Y|
|R14I3ZG5E6S7YM|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [11]:
# filter the dataframe and create a new dataframe to include all results where total_votes >= 20
vine_df1=vine_df.filter(vine_df.total_votes>=20)
vine_df1.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [12]:
# filter the dataframe and create a new dataframe including all results where helpful_votes/total_votes>=0.5
helpful_df = vine_df.filter(vine_df1.helpful_votes/vine_df1.total_votes >=0.5)
helpful_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2C99DJEO4RZ4K|          5|            3|          4|   N|                Y|
|R2DRL5NRODVQ3Z|          5|            2|          2|   N|                Y|
|R3T9B92MDDHKMM|          2|            5|          5|   N|                Y|
|R1OK6O3H5N02VG|          5|           11|         12|   N|                Y|
|R13S4RY124ZQB3|          3|            1|          1|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [13]:
# filter the dataframe and create a new dataframe including all results where a review was included as part of the vine program ("vine" == Y)
vine_reviews_df = helpful_df.filter(helpful_df.vine == "Y")
vine_reviews_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DQZOJNMI37KI|          4|            1|          2|   Y|                N|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|
|R2GTGOL5UM0TYI|          4|            4|          4|   Y|                N|
|R1A88KV72B9ZFZ|          5|            1|          2|   Y|                N|
|R2E9V8X9RJOVNR|          1|            2|          3|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [14]:
# filter the dataframe and create a new dataframe including all results where the review was NOT part of the vine program
not_vine_reviews_df = helpful_df.filter(helpful_df.vine == "N")
not_vine_reviews_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2C99DJEO4RZ4K|          5|            3|          4|   N|                Y|
|R2DRL5NRODVQ3Z|          5|            2|          2|   N|                Y|
|R3T9B92MDDHKMM|          2|            5|          5|   N|                Y|
|R1OK6O3H5N02VG|          5|           11|         12|   N|                Y|
|R13S4RY124ZQB3|          3|            1|          1|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



### Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the paid reviews

In [17]:
#total number of paid reviews
total_vine_reviews = vine_reviews_df.count()
total_vine_reviews

5028

In [18]:
# Total number of paid 5 star reviews
five_star_vine_reviews = vine_reviews_df.filter(vine_reviews_df.star_rating == 5).count()
five_star_vine_reviews

2368

In [21]:
# Percentage of 5 star reviews for paid reviews
five_star_vine_reviews/total_vine_reviews * 100

47.09626093874304

### Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the unpaid reviews

In [23]:
# total number of unpaid reviews
total_not_vine_reviews = not_vine_reviews_df.count()
total_not_vine_reviews

495942

In [22]:
#Total number of unpaid 5 star reviews
five_star_not_vine_reviews = not_vine_reviews_df.filter(not_vine_reviews_df.star_rating == 5).count()
five_star_not_vine_reviews

265843

In [24]:
# Percentage of 5 star reviews for unpaid reviews
five_star_not_vine_reviews/total_not_vine_reviews * 100

53.603647200680726